In [1]:
import numpy as np
import pandas as pd
from typing import List, Dict
from dataclasses import dataclass

In [2]:
test_timetable = pd.read_csv("test_graph.csv")

In [3]:
class Stop:
    def __init__(self, name: str, latitude: float, longitude: float) -> None:
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
    def __str__(self) -> str:
        return self.name + "[" + str(self.latitude) + ", " + str(self.longitude) + "]"
    def __repr__(self) -> str:
        return self.name + ", " + str(self.latitude) + "," + str(self.longitude)
    def __hash__(self) -> int:
        return hash(str(self.name))
    def __eq__(self, __value: object) -> bool:
        return isinstance(__value, Stop) and str(self.name) == str(__value.name)
    
@dataclass
class Route:
    line: str
    departure_time: str
    arrival_time: str

In [4]:
real_timetable = pd.read_csv("connection_graph (1).csv")

C:\Users\48791\AppData\Local\Temp\ipykernel_14628\3244852323.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  real_timetable = pd.read_csv("connection_graph (1).csv")


In [5]:
len(real_timetable)

996520

In [6]:
real_timetable.set_index(keys='Unnamed: 0', inplace=True)

In [9]:
real_timetable.drop_duplicates(ignore_index=True, inplace=True)

In [10]:
len(real_timetable)

456019

In [11]:
# timetable_dict: Dict[Stop, Route] = {}
# for stop in test_timetable.itertuples():
#     start_stop: Stop = Stop(stop.start_stop, stop.start_stop_lat, stop.start_stop_lon) 
#     end_stop: Stop = Stop(stop.end_stop, stop.end_stop_lat, stop.end_stop_lon)
#     end_destination: Route = Route(line=stop.line, destination=end_stop, departure_time=stop.departure_time, arrival_time=stop.arrival_time)
#     if start_stop not in timetable_dict.keys():
#         timetable_dict[start_stop] = [end_destination] 
#     else:
#         timetable_dict[start_stop].append(end_destination)

In [12]:
timetable_dict: Dict[Stop, Dict[Stop, List[Route]]] = {}
for stop in real_timetable.itertuples():
    start_stop: Stop = Stop(stop.start_stop, stop.start_stop_lat, stop.start_stop_lon) 
    end_stop: Stop = Stop(stop.end_stop, stop.end_stop_lat, stop.end_stop_lon)
    route: Route = Route(line=stop.line, departure_time=stop.departure_time, arrival_time=stop.arrival_time)
    if start_stop not in timetable_dict.keys():
        timetable_dict[start_stop] = {end_stop: [route]}
    elif end_stop in timetable_dict[start_stop].keys():
        timetable_dict[start_stop][end_stop].append(route)
    else:
        timetable_dict[start_stop][end_stop] = [route] 
    if end_stop not in timetable_dict.keys():
        timetable_dict[end_stop] = {}

In [13]:
@dataclass
class StopRecord():
    value: any
    last_stop: Stop
    last_route: Route

In [14]:
start = Stop("Tramwajowa", 51.10446678,17.08466997)
end = Stop("Muchobór Wielki", 51.09892535,16.94155277)
time = '00:53:00'
d_dict: Dict[Stop, StopRecord] = {} 
for key in timetable_dict.keys():
    d_dict[key] = StopRecord('99:99:99', None, None)
d_dict[start] = StopRecord(time, None, None)
q_set = list(timetable_dict.keys())
while(len(q_set) > 0):
    min_stop = q_set[0]
    for stop in q_set:
        if d_dict[stop].value < d_dict[min_stop].value:
            min_stop = stop
    q_set.remove(min_stop)
    for neighbor, routes in timetable_dict[min_stop].items():
        min_arrival_time = '99:99:99'
        min_arrival_id = -1
        for i in range(len(routes)):
            if d_dict[min_stop].value <= routes[i].departure_time and routes[i].arrival_time < min_arrival_time:
                min_arrival_time = routes[i].arrival_time
                min_arrival_id = i
        if d_dict[neighbor].value > min_arrival_time:
            d_dict[neighbor] = StopRecord(min_arrival_time, min_stop, routes[min_arrival_id])

In [15]:
for k,v in d_dict.items():
    print(k,'\t\t\t',v)

Zajezdnia Obornicka[51.14873744, 17.02106859] 			 StopRecord(value='07:54:00', last_stop=Paprotna, 51.14775215,17.02053929, last_route=Route(line=142, departure_time='07:53:00', arrival_time='07:54:00'))
Paprotna[51.14775215, 17.02053929] 			 StopRecord(value='05:22:00', last_stop=Bezpieczna, 51.14136001,17.02637623, last_route=Route(line=105, departure_time='05:21:00', arrival_time='05:22:00'))
Obornicka (Wołowska)[51.144385, 17.023735] 			 StopRecord(value='05:23:00', last_stop=Paprotna, 51.14775215,17.02053929, last_route=Route(line=111, departure_time='05:22:00', arrival_time='05:23:00'))
Bezpieczna[51.14136001, 17.02637623] 			 StopRecord(value='04:58:00', last_stop=Bałtycka, 51.13663187,17.0306168, last_route=Route(line=119, departure_time='04:57:00', arrival_time='04:58:00'))
Bałtycka[51.13663187, 17.0306168] 			 StopRecord(value='04:57:00', last_stop=Broniewskiego, 51.13585101,17.03738306, last_route=Route(line=119, departure_time='04:55:00', arrival_time='04:57:00'))
Broniewsk

In [16]:
d_dict[end]

StopRecord(value='05:00:00', last_stop=Zagony, 51.10087942,16.94224152, last_route=Route(line=152, departure_time='04:59:00', arrival_time='05:00:00'))

In [17]:
d_dict[d_dict[d_dict[end][1]][1]]

TypeError: 'StopRecord' object is not subscriptable

In [18]:
end, d_dict[end]

(Muchobór Wielki, 51.09892535,16.94155277,
 StopRecord(value='05:00:00', last_stop=Zagony, 51.10087942,16.94224152, last_route=Route(line=152, departure_time='04:59:00', arrival_time='05:00:00')))

In [19]:
def format_time(abnormal_time: str) -> str:
    return str(int(abnormal_time[:2])-24) + abnormal_time[2:] if int(abnormal_time[:2]) > 23 else abnormal_time

In [20]:
temp = end, d_dict[end]
route = []
max_length = [0,0,0,0,0]
while temp[1].last_stop is not None:
    route.append(temp)
    for i, element in enumerate([temp[1].last_route.line, temp[1].last_stop.name, format_time(temp[1].last_route.departure_time), temp[0].name, format_time(temp[1].last_route.arrival_time)]):
        max_length[i] = len(str(element)) if len(str(element)) > max_length[i] else max_length[i]
    temp = temp[1].last_stop, d_dict[temp[1].last_stop]
route.reverse()
for stop in route:
    print(f"{str(stop[1].last_route.line).rjust(max_length[0])}) [{format_time(stop[1].last_route.departure_time).rjust(max_length[2])}] {stop[1].last_stop.name.ljust(max_length[1])} - "
          f"[{format_time(stop[1].last_route.arrival_time)}] {stop[0].name}")

145) [04:11:00] Tramwajowa                           - [04:13:00] Hala Stulecia
145) [04:13:00] Hala Stulecia                        - [04:15:00] Kliniki - Politechnika Wrocławska
145) [04:15:00] Kliniki - Politechnika Wrocławska    - [04:17:00] PL. GRUNWALDZKI
145) [04:17:00] PL. GRUNWALDZKI                      - [04:18:00] most Grunwaldzki
145) [04:18:00] most Grunwaldzki                     - [04:20:00] Poczta Główna
145) [04:20:00] Poczta Główna                        - [04:22:00] skwer Krasińskiego
145) [04:22:00] skwer Krasińskiego                   - [04:24:00] DWORZEC GŁÓWNY
  9) [04:25:00] DWORZEC GŁÓWNY                       - [04:27:00] Wzgórze Partyzantów
 13) [04:29:00] Wzgórze Partyzantów                  - [04:30:00] Park Staromiejski
 13) [04:30:00] Park Staromiejski                    - [04:32:00] ŚWIDNICKA (Dom Europy)
 13) [04:32:00] ŚWIDNICKA (Dom Europy)               - [04:33:00] Świdnicka
 13) [04:33:00] Świdnicka                            - [04:34:00] Zamkowa


In [21]:
from datetime import time
time('2:07:00', format)

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
test_timetable.head

<bound method NDFrame.head of     Unnamed: 0       company  line departure_time arrival_time  \
0       107472  MPK Tramwaje     4       19:52:00     19:53:00   
1       107473  MPK Tramwaje     4       19:53:00     19:55:00   
2       107474  MPK Tramwaje     4       19:55:00     19:56:00   
3       107475  MPK Tramwaje     4       19:56:00     19:58:00   
4       107476  MPK Tramwaje     4       19:58:00     20:00:00   
5       107477  MPK Tramwaje     4       20:00:00     20:01:00   
6        56906  MPK Tramwaje     1       20:26:00     20:28:00   
7        56907  MPK Tramwaje     1       20:28:00     20:30:00   
8        56908  MPK Tramwaje     1       20:30:00     20:32:00   
9        56909  MPK Tramwaje     1       20:32:00     20:33:00   
10       56910  MPK Tramwaje     1       20:33:00     20:35:00   
11       56911  MPK Tramwaje     1       20:35:00     20:36:00   
12       56912  MPK Tramwaje     1       20:36:00     20:38:00   
13       56913  MPK Tramwaje     1       20:38

NameError: name 'Stop' is not defined

In [ ]:
test_arr

array([[107472, 'MPK Tramwaje', 4, '19:52:00', '19:53:00',
        'Chełmońskiego', 'Tramwajowa', 51.10388356, 17.09071195,
        51.10446678, 17.08466997],
       [107473, 'MPK Tramwaje', 4, '19:53:00', '19:55:00', 'Tramwajowa',
        'ZOO', 51.10446678, 17.08466997, 51.10556081, 17.0778982],
       [107474, 'MPK Tramwaje', 4, '19:55:00', '19:56:00', 'ZOO',
        'Hala Stulecia', 51.10556081, 17.0778982, 51.10708825,
        17.07346452],
       [107475, 'MPK Tramwaje', 4, '19:56:00', '19:58:00',
        'Hala Stulecia', 'Kliniki - Politechnika Wrocławska',
        51.10708825, 17.07346452, 51.10971261, 17.06511141],
       [107476, 'MPK Tramwaje', 4, '19:58:00', '20:00:00',
        'Kliniki - Politechnika Wrocławska', 'PL. GRUNWALDZKI',
        51.10971261, 17.06511141, 51.11144435, 17.05995961],
       [107477, 'MPK Tramwaje', 4, '20:00:00', '20:01:00',
        'PL. GRUNWALDZKI', 'most Grunwaldzki', 51.11144435, 17.05995961,
        51.11021216, 17.05538488],
       [56906, 'M

In [ ]:
for row in test_arr:
    stop = Stop(row[latitude])

NameError: name 'Stop' is not defined

In [ ]:
real_timetable = pd.read_csv("connection_graph (1).csv")

C:\Users\48791\AppData\Local\Temp\ipykernel_11972\3244852323.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  real_timetable = pd.read_csv("connection_graph (1).csv")


In [ ]:
real_timetable.head

<bound method NDFrame.head of         Unnamed: 0               company line departure_time arrival_time  \
0                0          MPK Autobusy    A       20:52:00     20:53:00   
1                1          MPK Autobusy    A       20:53:00     20:54:00   
2                2          MPK Autobusy    A       20:54:00     20:55:00   
3                3          MPK Autobusy    A       20:55:00     20:57:00   
4                4          MPK Autobusy    A       20:57:00     20:59:00   
...            ...                   ...  ...            ...          ...   
996515      996515  DLA Kąty Wrocławskie  967       18:38:00     18:39:00   
996516      996516  DLA Kąty Wrocławskie  967       18:39:00     18:41:00   
996517      996517  DLA Kąty Wrocławskie  967       18:41:00     18:42:00   
996518      996518  DLA Kąty Wrocławskie  967       18:42:00     18:43:00   
996519      996519  DLA Kąty Wrocławskie  967       18:43:00     18:45:00   

                         start_stop          

In [ ]:
print(real_timetable['arrival_time'].agg(['min', 'max']))

min    03:36:00
max    30:05:00
Name: arrival_time, dtype: object


In [ ]:
print("Values range for departure time: " + real_timetable['departure_time'].min() + " - " + real_timetable['departure_time'].max())
print("Values range for arrival time: " + real_timetable['arrival_time'].min() + " - " + real_timetable['arrival_time'].max())
print("Values range for start-stop latitude: " + str(real_timetable['start_stop_lat'].min()) + " - " + str(real_timetable['start_stop_lat'].max()))
print("Values range for start-stop longitude: " + str(real_timetable['start_stop_lon'].min()) + " - " + str(real_timetable['start_stop_lon'].max()))

Przedzial departure time: 03:34:00 - 30:03:00
Przedzial arrival time: 03:36:00 - 30:05:00
Przedzial start-stop latitude: 50.98042601 - 51.26556584
Przedzial start-stop longitude: 16.695853 - 17.2855453
